In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import tensorflow as tf
import keras_tuner as kt

<u> hyperparameter are of two types<u>:-
    
- 1) Model hyperparameter:- number of width of hidden layers    
- 2) algorithm hyperparameter:- learning rate for Stochastic Gradient Descent (SGD) and the number of nearest neighbors for a k Nearest Neighbors (KNN) classifier

In [5]:
(img_train, label_train), (img_test, label_test) = tf.keras.datasets.fashion_mnist.load_data()

In [10]:
img_train = img_train/255
img_test = img_test/255

In [15]:
img_train[0].shape

(28, 28)

In [32]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

In [33]:
def model_builder(hp):

    model = Sequential()
    model.add(Flatten(input_shape = (28,28)))

    # Tune the number of units in the first Dense layer
    # Choose an optimal value between 32-512

    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    model.add(Dense(units=hp_units, activation='relu'))
    model.add(Dense(10,activation= "softmax"))

    # Tune the learning rate for the optimizer
    # Choose an optimal value from 0.01, 0.001, or 0.0001

    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(Adam(learning_rate=hp_learning_rate),
                    loss= "SparseCategoricalCrossentropy",
                    metrics=['accuracy'])

    return model

In [34]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

In [38]:
stop_early = EarlyStopping(monitor='val_loss', patience=5)

In [39]:
tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

Trial 42 Complete [00h 00m 36s]
val_accuracy: 0.8884166479110718

Best val_accuracy So Far: 0.8926666378974915
Total elapsed time: 00h 30m 51s
INFO:tensorflow:Oracle triggered exit


In [37]:
# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")


The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 480 and the optimal learning rate for the optimizer
is 0.001.



In [40]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
1500/1500 [==============================] - 7s 5ms/step - loss: 0.4897 - accuracy: 0.8273 - val_loss: 0.3930 - val_accuracy: 0.8594
Epoch 2/50
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3688 - accuracy: 0.8657 - val_loss: 0.3486 - val_accuracy: 0.8775
Epoch 3/50
1500/1500 [==============================] - 7s 4ms/step - loss: 0.3305 - accuracy: 0.8796 - val_loss: 0.3428 - val_accuracy: 0.8748
Epoch 4/50
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3051 - accuracy: 0.8877 - val_loss: 0.3369 - val_accuracy: 0.8788
Epoch 5/50
1500/1500 [==============================] - 6s 4ms/step - loss: 0.2849 - accuracy: 0.8947 - val_loss: 0.3421 - val_accuracy: 0.8776
Epoch 6/50
1500/1500 [==============================] - 6s 4ms/step - loss: 0.2702 - accuracy: 0.8987 - val_loss: 0.3185 - val_accuracy: 0.8881
Epoch 7/50
1500/1500 [==============================] - 6s 4ms/step - loss: 0.2556 - accuracy: 0.9044 - val_loss: 0.3184 - val_accuracy:

In [41]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(img_train, label_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/36
1500/1500 [==============================] - 7s 4ms/step - loss: 0.4964 - accuracy: 0.8227 - val_loss: 0.3995 - val_accuracy: 0.8549
Epoch 2/36
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3694 - accuracy: 0.8662 - val_loss: 0.3964 - val_accuracy: 0.8508
Epoch 3/36
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3283 - accuracy: 0.8782 - val_loss: 0.3604 - val_accuracy: 0.8688
Epoch 4/36
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3052 - accuracy: 0.8866 - val_loss: 0.3476 - val_accuracy: 0.8769
Epoch 5/36
1500/1500 [==============================] - 6s 4ms/step - loss: 0.2834 - accuracy: 0.8951 - val_loss: 0.3138 - val_accuracy: 0.8869
Epoch 6/36
1500/1500 [==============================] - 6s 4ms/step - loss: 0.2692 - accuracy: 0.8994 - val_loss: 0.3247 - val_accuracy: 0.8854
Epoch 7/36
1500/1500 [==============================] - 6s 4ms/step - loss: 0.2526 - accuracy: 0.9058 - val_loss: 0.3183 - val_accuracy:

In [42]:
eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)

313/313 [==============================] - 1s 3ms/step - loss: 0.4959 - accuracy: 0.8894
[test loss, test accuracy]: [0.49594905972480774, 0.8894000053405762]


In [43]:
# official documentation ---> https://keras.io/keras_tuner/

----------